In [ ]:
%matplotlib inline
import holoviews as hv
hv.extension('bokeh', 'matplotlib',)
%opts Image[width=500, height=300, colorbar=True](cmap='magma') 
%opts QuadMesh[width=400,  height=200, colorbar=True](cmap='viridis')
%opts Curve[width=500, height=int(500/1.61)]

import matplotlib.pyplot as plt
plt.rcParams['savefig.dpi'] = 120
plt.style.use('presentation.mplstyle')
from gnl.colorblind import colorblind_matplotlib
colorblind_matplotlib()





import xarray as xr
from os.path import join, abspath
import os
from scipy import quantile


RUN = abspath(os.getenv('RUN', '../models/17/test'))
TRAINING = "../data/processed/training.nc"


data_2d_path = join(RUN, 'OUT_2D', '*.nc')
data_3d_path = join(RUN, 'OUT_3D', '*.nc')
data_stat_path = join(RUN, 'OUT_STAT', '*.nc')
no_nn_path = '/Users/noah/workspace/research/uwnet/data/runs/2018-09-27/OUT_3D/*.nc'


print(data_2d_path)
data_2d = xr.open_mfdataset(data_2d_path)
data_3d = xr.open_mfdataset(data_3d_path)
stat = xr.open_mfdataset(data_stat_path)
train_data = xr.open_dataset(TRAINING)
no_nn = xr.open_mfdataset(no_nn_path)

data_3d['FQTNN'] *= 86400
data_3d['FQTNN'].attrs['units'] = 'g/kg/d'
data_3d['FSLINN'] *= 86400
data_3d['FSLINN'].attrs['units'] = 'K/d'

In [ ]:
! [ -d gmu-figs ] || mkdir gmu-figs

In [ ]:
data_2d.PW[::22].plot(col='time', col_wrap=3)

In [ ]:
data_2d.W500[::22].plot(col='time', col_wrap=3)

In [ ]:
data_3d.W[::7].mean('x').plot(col='time', col_wrap=3)

# Forcing in a Single Location

In [ ]:
loc = dict(x=0, y=32)

data_3d['FQTNN'].isel(**loc).plot.contourf(x='time', levels=11, vmax=10)
plt.figure()
data_3d['FSLINN'].isel(**loc).plot.contourf(x='time', levels=11)

The training data is sampled twice as frequently. Let's align the two

In [ ]:
train_data_resampled = train_data.sel(time=data_3d.time, method='nearest')\
.assign_coords(x=data_3d.x, y=data_3d.y, time=data_3d.time)

no_nn_resampled = no_nn.sel(time=data_3d.time, method='nearest')\
.assign_coords(x=data_3d.x, y=data_3d.y, time=data_3d.time)

## Spatial structure of the error

In [ ]:
forecast_error = data_3d - train_data_resampled

qt_error = train_data_resampled.QT -data_3d.QT

sli_error = train_data_resampled.SLI -data_3d.SLI
u_error = train_data_resampled.U -data_3d.U

u_error_persistence = train_data_resampled.U[0] - train_data_resampled.U

## Comparison with persistence and null forecasts

In [ ]:
def rms(x):
    return np.sqrt((x**2).mean(['x', 'y']))

def error_plot(forecast, no_nn, truth, unit=''):
    error  = rms(forecast- truth)
    persistence_error = rms(truth[0] - truth)
    time = error.time
    plt.plot(time,error, label='NN')
    plt.plot(time,persistence_error, label='Persistence Forecast')
    plt.plot(time,rms(no_nn-truth), label='No NN')
    plt.xlabel('day')
    plt.ylabel(unit)
    plt.legend()
    
def plot_err_height_field(field, level, **kw):
    error_plot(data_3d[field][:,level], no_nn_resampled[field][:,level], train_data_resampled[field][:,level], **kw)
    plt.title(f"{field} RMS Error at z={float(data_3d.z[level])}")
    plt.tight_layout()

    
def plot_err_field(field):
    levs = [5,10,15,20]

    for lev in levs:
        plt.figure()
        plot_err_height_field(field, lev)

In [ ]:
plot_err_height_field('SLI', 15, unit='K')
plt.savefig('gmu-figs/sli-err.pdf')
plt.tight_layout()

In [ ]:
plot_err_height_field('QT', 10, unit='g/kg')
plt.savefig('gmu-figs/qt-err.pdf')

In [ ]:
plot_err_height_field('U', 15, unit='m/s')
plt.savefig('gmu-figs/u-err.pdf')

In [ ]:
plot_err_height_field('W', 15, unit='m/s')
plt.savefig('gmu-figs/w-err.pdf')

# Error in P-E

In [ ]:
net_precip_nn = -(data_3d.FQTNN/1000 * train_data.layer_mass).sum('z')
t_2d_plot= net_precip_nn[::5].time

net_precip_obs = train_data.Prec - train_data.LHF/2.51e6 * 86400


In [ ]:
fig, (obs, nn) = plt.subplots(1, 2, constrained_layout=True, figsize=(10,4))

t = 101.875

im = net_precip_obs.sel(time=t).plot(vmax=50, ax=obs, add_labels=False, add_colorbar=False)
net_precip_nn.sel(time=t).plot(vmax=50, ax=nn, add_labels=False, add_colorbar=False)

plt.colorbar(im, ax=[obs,nn], label='mm/day')
obs.set_title("NG-Aqua (obs)")
nn.set_title("NN+SAM")

plt.savefig('gmu-figs/net-precip-snapshot.png')

In [ ]:
net_precip_nn[10].plot()

In [ ]:
net_precip_nn.mean('x')[-1].plot(label='NN+SAM')
net_precip_obs.mean(['x', 'time']).plot(label='OBS')
plt.legend()
plt.title("Zonal mean net precipitation (mm/day)")
plt.tight_layout()
plt.savefig("gmu-figs/net-precip.pdf")


### Equatorial winds

Let's look at the difference in the zonal mean zonal velocity at the equator in NGAqua and in the truth.

In [ ]:
nn = data_3d.U.mean('x').isel(y=32)
obs = train_data_resampled.U.isel(y=32).mean('x')

In [ ]:
%%opts Curve{+framewise}
hv.Dataset(obs.load()).to.curve("time", label="SAM") \
   * hv.Dataset(nn.load()).to.curve("time", label="NN")

The surface easterlies in SAM/NN are much smaller than in NGAqua, but are larger in magnitude above 1km. As Chris suggests, this could indicate there is not enough vertical momentum mixing in the tropics of this model. This mixing could be increased by a good parametrization of CMT.

# Tarball up the figures

In [ ]:
!tar czf gmu-figs.tgz gmu-figs/